In [1]:
# Working example
import pandas as pd
from cytominer_eval import evaluate

# Load Data
path = "profiles/NCP_PILOT_3/BR_NCP_PILOT_3/BR_NCP_PILOT_3_normalized_variable_selected.csv.gz"

df = pd.read_csv(path)

# Define important function arguments
meta_features = df.columns[df.columns.str.startswith("Metadata_")]
features = df.drop(meta_features, axis="columns").columns.tolist()


In [2]:
df.head(5)

,Metadata_Plate,Metadata_Well,Metadata_Assay_Plate_Barcode,Metadata_Plate_Map_Name,Metadata_well_position,Metadata_plating_density,Metadata_line_condition,Metadata_compound_ID,Metadata_replicate,Cellbodies_AreaShape_Compactness,...,Neurites_Texture_InfoMeas2_RNA_20_03,Neurites_Texture_InverseDifferenceMoment_AGP_10_02,Neurites_Texture_InverseDifferenceMoment_AGP_20_02,Neurites_Texture_InverseDifferenceMoment_DNA_5_00,Neurites_Texture_SumAverage_AGP_10_00,Neurites_Texture_SumAverage_RNA_20_00,Neurites_Texture_SumVariance_Brightfield_10_03,Neurites_Texture_SumVariance_DNA_20_03,Neurites_Texture_SumVariance_ER_20_03,Neurites_Texture_Variance_AGP_5_02
0,BR_NCP_PILOT_3,A01,BR_NCP_PILOT_3,BR_NCP_PILOT_3,A01,2500,control,Untreated,control_Untreated,-1.2745,...,0.68736,0.32353,0.38678,-1.333400,0.84262,0.43418,2.85160,-0.065548,1.05590,1.8216
1,BR_NCP_PILOT_3,A02,BR_NCP_PILOT_3,BR_NCP_PILOT_3,A02,10000,control,Untreated,control_Untreated,-1.1154,...,1.42290,-1.49340,-1.79440,-0.053887,1.81270,2.02500,4.02830,-0.377890,1.38140,2.0071
2,BR_NCP_PILOT_3,A03,BR_NCP_PILOT_3,BR_NCP_PILOT_3,A03,2500,control,BRD-K79131256-001-17-9,control_BRD-K79131256-001-17-9,-1.3822,...,0.83461,-0.84041,-0.30974,0.854620,1.29650,0.03451,0.90154,-0.094023,0.67013,2.7642
3,BR_NCP_PILOT_3,A04,BR_NCP_PILOT_3,BR_NCP_PILOT_3,A04,10000,control,BRD-K36207157-001-09-6,control_BRD-K36207157-001-09-6,-1.5901,...,1.66050,-1.73520,-1.53300,-0.472280,1.98300,1.34460,1.43450,0.477740,1.00020,2.0469
4,BR_NCP_PILOT_3,A05,BR_NCP_PILOT_3,BR_NCP_PILOT_3,A05,2500,control,Untreated,control_Untreated,-1.0079,...,2.11990,1.42280,1.51170,0.554710,-0.27509,-0.17327,0.92286,-0.267170,0.50922,1.2480


In [3]:
df.sample(5)

,Metadata_Plate,Metadata_Well,Metadata_Assay_Plate_Barcode,Metadata_Plate_Map_Name,Metadata_well_position,Metadata_plating_density,Metadata_line_condition,Metadata_compound_ID,Metadata_replicate,Cellbodies_AreaShape_Compactness,...,Neurites_Texture_InfoMeas2_RNA_20_03,Neurites_Texture_InverseDifferenceMoment_AGP_10_02,Neurites_Texture_InverseDifferenceMoment_AGP_20_02,Neurites_Texture_InverseDifferenceMoment_DNA_5_00,Neurites_Texture_SumAverage_AGP_10_00,Neurites_Texture_SumAverage_RNA_20_00,Neurites_Texture_SumVariance_Brightfield_10_03,Neurites_Texture_SumVariance_DNA_20_03,Neurites_Texture_SumVariance_ER_20_03,Neurites_Texture_Variance_AGP_5_02
145,BR_NCP_PILOT_3,G02,BR_NCP_PILOT_3,BR_NCP_PILOT_3,G02,10000,deletion,Untreated,deletion_Untreated,-0.042634,...,-0.479250,-0.75444,-0.837660,-1.74080,-0.43452,0.58393,0.13729,2.21420,-0.506310,-0.55036
365,BR_NCP_PILOT_3,P06,BR_NCP_PILOT_3,BR_NCP_PILOT_3,P06,7500,deletion,BRD-K21680192-300-14-4,deletion_BRD-K21680192-300-14-4,-2.442800,...,-2.063200,-0.25385,-0.052359,-0.24959,0.68996,-1.63170,1.08330,-0.69614,-1.393500,-0.25014
327,BR_NCP_PILOT_3,N16,BR_NCP_PILOT_3,BR_NCP_PILOT_3,N16,7500,control,BRD-K15108141-003-04-7,control_BRD-K15108141-003-04-7,0.526180,...,1.737300,0.69147,0.397910,-0.42456,0.58911,2.43910,-0.29555,2.75070,3.513200,0.09998
194,BR_NCP_PILOT_3,I03,BR_NCP_PILOT_3,BR_NCP_PILOT_3,I03,2500,control,BRD-K79131256-001-17-9,control_BRD-K79131256-001-17-9,-1.087100,...,0.047024,-1.51710,-1.113500,1.22060,1.32390,-0.88505,-0.92400,-1.07170,-0.043427,1.11700
149,BR_NCP_PILOT_3,G06,BR_NCP_PILOT_3,BR_NCP_PILOT_3,G06,10000,deletion,Untreated,deletion_Untreated,0.231360,...,-0.553020,0.67650,0.232420,-1.91430,-0.37280,1.43060,1.17620,4.00720,-0.338180,-1.03570


In [4]:
# Evaluate profile quality using grit

control_perts = [
    "control_Untreated"
]

replicate_groups = {
    "replicate_id": "Metadata_replicate",
    "group_id": "Metadata_Plate",
}

grit_results_df = evaluate(
    profiles=df,
    features=features,
    meta_features=meta_features,
    replicate_groups=replicate_groups,
    operation="grit",
    grit_control_perts=control_perts,
)

grit_results_df

,perturbation,group,grit
0,control_BRD-K15108141-003-04-7,BR_NCP_PILOT_3,-0.510551
1,control_BRD-K21680192-300-14-4,BR_NCP_PILOT_3,-0.237017
2,control_BRD-K36207157-001-09-6,BR_NCP_PILOT_3,-0.774204
3,control_BRD-K79131256-001-17-9,BR_NCP_PILOT_3,-0.656651
4,control_Untreated,BR_NCP_PILOT_3,-0.971370
5,deletion_BRD-K15108141-003-04-7,BR_NCP_PILOT_3,0.556986
6,deletion_BRD-K21680192-300-14-4,BR_NCP_PILOT_3,-0.132433
7,deletion_BRD-K36207157-001-09-6,BR_NCP_PILOT_3,0.510883
8,deletion_BRD-K79131256-001-17-9,BR_NCP_PILOT_3,0.586947
9,deletion_Untreated,BR_NCP_PILOT_3,0.237042


In [5]:
# But the grouping we actually want is by Metadata_plating_density
replicate_groups = {
    "replicate_id": "Metadata_replicate",
    "group_id": "Metadata_plating_density",
}

grit_results_df = evaluate(
    profiles=df,
    features=features,
    meta_features=meta_features,
    replicate_groups=replicate_groups,
    operation="grit",
    grit_control_perts=control_perts,
)

grit_results_df

AssertionError: grit is calculated for each perturbation independently